In [12]:
import cv2
from ultralytics import YOLO

def counting_cards(class_names: list) -> int:
    class_pred = [card[:-1] for card in class_names]

    num_list = ['2', '3', '4', '5', '6', '7', '8', '9', '10']
    ten_list = ['J', 'K', 'Q']

    rs = 0  

    for i in range(len(class_pred)):
        if class_pred[i] in num_list:
            rs+=int(class_pred[i])
        elif class_pred[i] in ten_list:
            rs+=10
        elif class_pred[i]=='A':
            if rs<21:
                rs+=11
            else:
                rs+=1

    return rs

class Pipeline:
    def __init__(self, path_model: str):
        self.model = YOLO(path_model)

    def train_model(self, save_path, data_path, ep=50, batch_size=8,optomizer_name = 'Adam', size_fr = 8):
        self.model.train(
            data=data_path,    
            epochs=ep,            
            batch=batch_size,            
            project='runs/train', 
            name=save_path, 
            optomizer = optomizer_name,
            freeze = size_fr
        )

    def detection_cards(self):
        cap_1 = cv2.VideoCapture(0)

        if not cap_1.isOpened():
            print("Не удалось открыть камерy.")
            exit()

        while True:
            ret1, frame1 = cap_1.read()
            if not ret1 :
                print("Ошибка при чтении камеры.")
                break

   
            results_1 = self.model(frame1)
            annotated_frame_1 = results_1[0].plot() 
            font = cv2.FONT_HERSHEY_SCRIPT_COMPLEX     

            cv2.imshow('Детекция карт_1', annotated_frame_1)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap_1.release()
        cv2.destroyAllWindows()

    def black_jack_game(self):
        cap_1 = cv2.VideoCapture(0)
        cap_2 = cv2.VideoCapture(1)

        if not cap_1.isOpened() or not cap_2.isOpened():
            print("Не удалось открыть одну из камер.")
            exit()

        # Шрифт для вывода текста
        font = cv2.FONT_HERSHEY_SIMPLEX

        current_mode = 0
        while True:
            ret1, frame1 = cap_1.read()
            ret2, frame2 = cap_2.read()
            if not ret1 or not ret2:
                print("Ошибка при чтении с одной из камер.")
                break

    # Применяем модель к кадрам
            results_1 = self.model(frame1)
            results_2 = self.model(frame2)

    # Для камеры 1
            class_names_1 = [self.model.names[int(box.cls[0])] for box in results_1[0].boxes]  # Имена классов для каждого объекта
            annotated_frame_1 = results_1[0].plot()  # Получаем изображение с аннотациями
            class_names_1 = list(set(class_names_1))
            res_1  = counting_cards( class_names_1)

    # Для камеры 2
            class_names_2 = [self.model.names[int(box.cls[0])] for box in results_2[0].boxes]  # Имена классов для каждого объекта
            annotated_frame_2 = results_2[0].plot()  # Получаем изображение с аннотациями
            class_names_2 = list(set(class_names_2))
            res_2  = counting_cards( class_names_2)


            status_1 = ''
            status_2 = ''
            if (res_1>res_2 and res_1<21) or (res_1 < res_2 and res_2>21) or (res_1==21 and res_2!=21):
                status_1 = 'Win'
                status_2 = 'Lose'
            elif (res_2>res_1 and res_2<21) or (res_2<res_1 and res_1>21) or (res_2==21 and res_1!=21):
                status_1 = 'Lose'
                status_2 = 'Win'
            elif res_1>21 and res_2>21:
                if res_1<res_2:
                    status_1 = 'Win'
                    status_2 = 'Lose'
                else:
                    status_1 = 'Lose'
                    status_2 = 'Win'
            if res_1==res_2:
                status_1 = 'Push'
                status_2 = 'Push'
            
            if current_mode==0:
                cv2.putText(frame1, f'Dealer: {", ".join(class_names_1)} -> {res_1}', (10, 30), font, 1, (255, 255, 255), 1, cv2.LINE_AA)
                cv2.putText(frame2, f'Player: {", ".join(class_names_2)} -> {res_2}', (10, 30), font, 1, (255, 255, 255), 1, cv2.LINE_AA)

                cv2.imshow('Детекция карт_1 (Dealer)', frame1)
                cv2.imshow('Детекция карт_2 (Player)', frame2)
            elif current_mode==1:
                cv2.putText(annotated_frame_1, f'Dealer: {", ".join(class_names_1)} -> {res_1}', (10, 30), font, 1, (255, 255, 255), 1, cv2.LINE_AA)
                cv2.putText(annotated_frame_2, f'Player: {", ".join(class_names_2)} -> {res_2}', (10, 30), font, 1, (255, 255, 255), 1, cv2.LINE_AA)

                cv2.imshow('Детекция карт_1 (Dealer)', annotated_frame_1)
                cv2.imshow('Детекция карт_2 (Player)', annotated_frame_2)
            elif current_mode==2:
                cv2.putText(frame1, f'Dealer: {res_1}->{status_1}', (10, 30), font, 1, (255, 255, 255), 1, cv2.LINE_AA)
                cv2.putText(frame2, f'Player: {res_2}->{status_2}', (10, 30), font, 1, (255, 255, 255), 1, cv2.LINE_AA)

                cv2.imshow('Детекция карт_1 (Dealer)', frame1)
                cv2.imshow('Детекция карт_2 (Player)', frame2)
        
            key = cv2.waitKey(1) & 0xFF
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            elif cv2.waitKey(1) & 0xFF == ord('a'):
                current_mode = 1
            elif cv2.waitKey(1) & 0xFF == ord('p'):
                current_mode = 2

        cap_1.release()
        cap_2.release()
        cv2.destroyAllWindows()

game.black_jack_game()

In [13]:
game = Pipeline(path_model='runs/train/yolov10s_8layers_top(main_2)/weights/best.pt')

In [ ]:
game.detection_cards()

In [ ]:
game.black_jack_game()


0: 384x640 1 2C, 2 9Hs, 2 JDs, 14.0ms
Speed: 3.0ms preprocess, 14.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 480x640 1 4D, 1 4S, 2 6Cs, 17.0ms
Speed: 4.0ms preprocess, 17.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 384x640 1 2C, 2 9Hs, 2 JDs, 16.0ms
Speed: 2.0ms preprocess, 16.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 480x640 1 4D, 1 4S, 2 6Cs, 17.0ms
Speed: 2.0ms preprocess, 17.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 384x640 1 2C, 2 9Hs, 2 JDs, 23.0ms
Speed: 3.0ms preprocess, 23.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 480x640 1 4D, 1 4S, 2 6Cs, 22.0ms
Speed: 4.0ms preprocess, 22.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 384x640 1 2C, 2 9Hs, 2 JDs, 19.0ms
Speed: 2.0ms preprocess, 19.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 480x640 1 4D, 1 4S, 2 6Cs, 19.0ms
Speed: 2.0ms 